# Rf 3 0 3_Conditional
Multidimensional models: use of tailored p.d.f as conditional p.d.fs.s

 pdf = gauss(x,f(y),sx | y ) with f(y) = a0 + a1*y



**Author:** 07/2008 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:47 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooDataHist.h"
#include "RooGaussian.h"
#include "RooPolyVar.h"
#include "RooProdPdf.h"
#include "RooPlot.h"
#include "TRandom.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "TH1.h"

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

In [3]:
RooDataSet *makeFakeDataXY();

input_line_28:2:28: warning: empty parentheses interpreted as a function declaration [-Wvexing-parse]
 RooDataSet *makeFakeDataXY();
                           ^~
input_line_28:2:28: note: replace parentheses with an initializer to declare a variable
 RooDataSet *makeFakeDataXY();
                           ^~
                            = nullptr


 A helper function is created: 

In [4]:
%%cpp -d
RooDataSet *makeFakeDataXY()
{
   RooRealVar x("x", "x", -10, 10);
   RooRealVar y("y", "y", -10, 10);
   RooArgSet coord(x, y);

   RooDataSet *d = new RooDataSet("d", "d", RooArgSet(x, y));

   for (int i = 0; i < 10000; i++) {
      Double_t tmpy = gRandom->Gaus(0, 10);
      Double_t tmpx = gRandom->Gaus(0.5 * tmpy, 1);
      if (fabs(tmpy) < 10 && fabs(tmpx) < 10) {
         x = tmpx;
         y = tmpy;
         d->add(coord);
      }
   }

   return d;
}

Setup composed model gauss(x,m(y),s)
 -----------------------------------------------------------------------

Create observables

In [5]:
RooRealVar x("x", "x", -10, 10);
RooRealVar y("y", "y", -10, 10);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Create function f(y) = a0 + a1*y

In [6]:
RooRealVar a0("a0", "a0", -0.5, -5, 5);
RooRealVar a1("a1", "a1", -0.5, -1, 1);
RooPolyVar fy("fy", "fy", y, RooArgSet(a0, a1));

Create gauss(x,f(y),s)

In [7]:
RooRealVar sigma("sigma", "width of gaussian", 0.5, 0.1, 2.0);
RooGaussian model("model", "Gaussian with shifting mean", x, fy, sigma);

Obtain fake external experimental dataset with values for x and y

In [8]:
RooDataSet *expDataXY = makeFakeDataXY();

Generate data from conditional p.d.f model(x|y)
 ---------------------------------------------------------------------------------------------

Make subset of experimental data with only y values

In [9]:
RooDataSet *expDataY = (RooDataSet *)expDataXY->reduce(y);

Generate 10000 events in x obtained from _conditional_ model(x|y) with y values taken from experimental data

In [10]:
RooDataSet *data = model.generate(x, ProtoData(*expDataY));
data->Print();

RooDataSet::modelData[x,y] = 6850 entries


Fit conditional p.d.f model(x|y) to data
 ---------------------------------------------------------------------------------------------

In [11]:
model.fitTo(*expDataXY, ConditionalObservables(y));

[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 a0          -5.00000e-01  1.00000e+00   -5.00000e+00  5.00000e+00
     2 a1          -5.00000e-01  2.00000e-01   -1.00000e+00  1.00000e+00
     3 sigma        5.00000e-01  1.90000e-01    1.00000e-01  2.00000e+00
 **********
 **    3 **SET ERR         0.5
 **********
 **********
 **    4 **SET PRINT           1
 **********
 **********
 **    5 **SET STR           1
 **********
 NOW USING STRATEGY  1: TRY TO BALANCE SPEED AGAINST RELIABILITY
 **********
 **    6 **MIGRAD        1500           1
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
 START MIGRAD MINIMIZATION.  STRATEGY  1.  CONVERGENCE WHEN EDM .LT. 1.00e-03
 FCN=421420 FROM MIGRAD    STATUS=INITIATE       12 CALLS          13 TOTAL


Project conditional p.d.f on x and y dimensions
 ---------------------------------------------------------------------------------------------

Plot x distribution of data and projection of model on x = 1/ndata sum(data(y_i)) model(x;y_i)

In [12]:
RooPlot *xframe = x.frame();
expDataXY->plotOn(xframe);
model.plotOn(xframe, ProjWData(*expDataY));

[#1] INFO:Plotting -- RooAbsReal::plotOn(model) plot on x averages using data variables (y)
[#1] INFO:Plotting -- RooDataWeightedAverage::ctor(modelDataWgtAvg) constructing data weighted average of function model_Norm[x] over 6850 data points of (y) with a total weight of 6850
.........................................................................................................................................................................................................

Speed up (and approximate) projection by using binned clone of data for projection

In [13]:
RooAbsData *binnedDataY = expDataY->binnedClone();
model.plotOn(xframe, ProjWData(*binnedDataY), LineColor(kCyan), LineStyle(kDotted));


[#1] INFO:Plotting -- RooAbsReal::plotOn(model) plot on x averages using data variables (y)
[#1] INFO:Plotting -- RooDataWeightedAverage::ctor(modelDataWgtAvg) constructing data weighted average of function model_Norm[x] over 100 data points of (y) with a total weight of 6850
.........................................................................................................................................................................................................

Show effect of projection with too coarse binning

In [14]:
((RooRealVar *)expDataY->get()->find("y"))->setBins(5);
RooAbsData *binnedDataY2 = expDataY->binnedClone();
model.plotOn(xframe, ProjWData(*binnedDataY2), LineColor(kRed));


[#1] INFO:Plotting -- RooAbsReal::plotOn(model) plot on x averages using data variables (y)
[#1] INFO:Plotting -- RooDataWeightedAverage::ctor(modelDataWgtAvg) constructing data weighted average of function model_Norm[x] over 5 data points of (y) with a total weight of 6850
.................................................................................................................................................................................................................................................

Make canvas and draw rooplots

In [15]:
new TCanvas("rf303_conditional", "rf303_conditional", 600, 460);
gPad->SetLeftMargin(0.15);
xframe->GetYaxis()->SetTitleOffset(1.2);
xframe->Draw();

Draw all canvases 

In [16]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()